In [ ]:
import requests
import json
import pandas as pd
import time
import traceback

In [ ]:
performance_api_url = "https://performance-reports-api.rankmi.com"

## Autenticación

Realizamos la autenticación con token de usuario previamente extraido de la configuración de Falabella

In [ ]:
secret_key = "SECRET-KEY-HERE"

In [ ]:
query = """mutation SignIn {
  signIn(input:{
    secretKey:"%s",
    clientMutationId:""
  }) {
    jwt
    errors
    code
  }
}""" % secret_key

In [ ]:
url = "{}/authentication/sign_in".format(performance_api_url)

In [ ]:
r = requests.post(url, json={'query':query})

if r.status_code != 200:
    raise ValueError("Error inesperado de Autenticacion: {}".format(r.text))

json_data = json.loads(r.text)
jwt = json_data["data"]["signIn"]["jwt"] if 'data' in json_data and 'signIn' in json_data['data'] else None
    
if jwt is None:
    raise ValueError("Error de Autenticacion: {}".format(json_data["data"]["signIn"]["errors"]))

headers = { "Authorization" : jwt }

## Proceso: Evaluación Ejecutiva 2020

In [ ]:
process = "PROCESS-TOKEN-HERE"

## Resumen de colaboradores

Ahora con el listado de colaboradores, iteraremos para extraer toda la información de resultados individuales y con esta información armar el dataframe con el resumen de la información requerida. Para realizar las pruebas de un ejercicio, tomaremos solo un numero limitado de registros (N_COLLABORATORS) y construiremos la data conforme el requerimiento de Falabella para este proceso. 

In [ ]:
url_resume_res = "{}/reports/resume".format(performance_api_url)

def get_resume_results(proceso, page=1):
    query_ind_result = """query {
      collaboratorsResume(
        processToken: "%s"
        page: %d
        lang: "es"
      ) {
        id
        rut
        name
        token
        avatar
        email
        position
        ponderationFamily
        talent
        process {
          id
          token
          name
        }
        areas {
          id
          parentAreaId
          name
          main
        }
        demographics {
          id
          name
          value
        }
        domains {
          domainId
          name
          domainAverage
          categoriesConducts
        }
      }
      pagination {
        currentPage
        limitValue
        totalPages
        totalRecords
      }
    }""" % (proceso, page)
    
    rindividual = requests.post(url_resume_res, json={'query':query_ind_result}, headers=headers)
    
    if rindividual.status_code != 200:
        raise ValueError("Error inesperado de Consulta Resumida de Colaboradores:", rindividual.text)

    json_individ = json.loads(rindividual.text)

    collaboratorsResume = json_individ['data']['collaboratorsResume'] if 'data' in json_individ and 'collaboratorsResume' in json_individ['data'] else None

    if collaboratorsResume is None:
        raise ValueError("Error de Consulta Resumida de Colaboradores: {}".format(json_individ['errors'][0]['message']))
        
    return collaboratorsResume, json_individ["data"]["pagination"]["totalPages"]

### Constantes & Catalogos

In [ ]:
MAX_PAGES = 5

CATEGORY = {
    'Parcial fulfils':'Cumple Satisfactorio',
    'Satisfactory fulfils':'Sobresaliente',
    'Under expected':'Bajo lo esperado',
    'Outstanding':'Pendiente',
    'Maintain role':'Mantener rol actual',
    'Expand Respons.':'Ampliar responsabilidad',
    'Grow 1 level':'Crecer un nivel',
    'Value':'Valor',
    'Under development':'Bajo Desarrollo',
    'Expert':'Experto',
    'Solid +':'Sólido +'
}

RESULT_TYPE = {
    'Performance':'Desempeño',
    'Development potential':'Potencial de desarrollo'
}

CALIBRATION = {
    True:'Calibrado',
    False:'Aceptada',
}

UNKNOWED_DATA = '---'
NULL_DATA = '*'
EMPTY_DATA = ''

### Mapping & Format

In [ ]:
def get_domains(r_ind, base_data):
    domains = r_ind['domains'] if 'domains' in r_ind else None
    domains_sumary = []
    
    if domains and len(domains) > 0:
        for d in domains:
            dAv, competences = d['domainAverage'], d['categoriesConducts']
            
            if not dAv: break
            
            for comp in competences:
                data =  base_data.copy()
                data['Dominio'] = d["name"]
                data['Grupo Competencia'] = comp["competenceName"]
                data['Competencia'] = comp["name"]
                data['Nota'] = comp["average"]
                data['Categoria'] = comp["label"]
                
                domains_sumary.append(data)
        
    return domains_sumary

def build_demographics(demo):
    demo_json = {}
    for item in demo:
        demo_json[item['name']] = item['value']
    return demo_json
    
def get_areas(areas):
    areas_json, index = {}, 1
    for area in areas:
        if area["main"]: areas_json['principal'] = area['name']
        areas_json['level{}'.format(index)] = area['name']
        index+=1
    return areas_json

### Procesamiento del listado de colaboradores para buscar resultados individuales

In [ ]:
indiv_data_collaborators = []

start = time.time()
total_pages = MAX_PAGES

for page in range(1, MAX_PAGES+1):
    print("-> Running page %d..." % page)
    
    if page > total_pages:
        print("<- Max Pages Reached")
        break
    
    results, total_pages = get_resume_results(process, page)

    for r_ind in results:
        try:
            demo = build_demographics(r_ind['demographics'])
            areas = get_areas(r_ind['areas'])
            talent = r_ind['talent'] if 'talent' in r_ind else None

            base_data = {
                'ID Proceso':r_ind['process']['id'],
                'Nombre Proceso':r_ind['process']['name'],
                'Identifier Colaborador':r_ind["rut"],
                'Nombre Colaborador':r_ind["name"],
                'Correo Colaborador':r_ind["email"],
                'Cargo Colaborador':r_ind["position"],
                'Familia de Cargo':demo["Familia de cargo"] if "Familia de cargo" in demo else EMPTY_DATA,
                'Genero Colaborador':demo["Genero Colaborador"] if "Genero" in demo else EMPTY_DATA,
                'Fecha de Nacimiento':demo["Fecha de Nacimiento"] if "Fecha de Nacimiento" in demo else EMPTY_DATA,
                'Ingreso Cargo':demo["Ingreso Cargo"] if "Ingreso Cargo" in demo else EMPTY_DATA,
                'Ingreso Grupo':demo["Ingreso Grupo"] if "Ingreso Grupo" in demo else EMPTY_DATA,
                'Formulario':demo["Formulario"] if "Formulario" in demo else EMPTY_DATA,
                'Talento Critico?': ("SI" if "iscritical" in talent and talent["iscritical"] else "NO") if talent else EMPTY_DATA,
                'Etiqueta Calibracion':UNKNOWED_DATA,
                'Tienda Central':demo["Tienda Central"] if "Tienda Central" in demo else EMPTY_DATA,
                'País':demo["País"] if "País" in demo else EMPTY_DATA,
                'Negocio':demo["Negocio"] if "Negocio" in demo else EMPTY_DATA,
                'Empresa':demo["Empresa"] if "Empresa" in demo else EMPTY_DATA,
                'Area Transversal':demo["Area Transversal"] if "Area Transversal" in demo else EMPTY_DATA,
                'Area Final':areas['principal'] if 'principal' in areas else EMPTY_DATA,
                'Nivel 1':areas["level1"] if "level1" in areas else EMPTY_DATA,
                'Nivel 2':areas["level2"] if "level2" in areas else EMPTY_DATA,
                'Nivel 3':areas["level3"] if "level3" in areas else EMPTY_DATA,
                'Nivel 4':areas["level4"] if "level4" in areas else EMPTY_DATA,
                'Nivel 5':areas["level5"] if "level5" in areas else EMPTY_DATA,
                'Nivel 6':areas["level6"] if "level6" in areas else EMPTY_DATA,
                'Nivel 7':areas["level7"] if "level7" in areas else EMPTY_DATA,
                'Nivel 8':areas["level8"] if "level8" in areas else EMPTY_DATA,
                'Tipo de Evaluacion':r_ind["ponderationFamily"] if "ponderationFamily" in r_ind else EMPTY_DATA
            }

            domains = get_domains(r_ind, base_data)
            indiv_data_collaborators.extend(domains)
        except Exception as e:
            print(e)
            traceback.print_exc()

print("Lapsed Time:", (time.time()-start))

## Resultados de: Ejecutivo Dominio

Desplegamos en un dataframe y exportamos a excel (el dataframe solo puede mostrar 50 elementos)

In [ ]:
collaborators_df = pd.DataFrame(indiv_data_collaborators)
collaborators_df

In [ ]:
# determining the name of the file
file_name = 'EjecutivoCompetencias.xlsx'
  
# saving the excel
collaborators_df.to_excel(file_name)

print("File saved here:", file_name)